In [0]:
import pandas as pd
!pip install tensorflow==2.0.0

import tensorflow as tf
print(tf.__version__)

import nltk
import gensim
import numpy as np
from gensim import corpora, models, similarities
from keras.utils import to_categorical
from numpy import argmax
nltk.download('punkt')
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Softmax, LSTM, Embedding
#tf.enable_eager_execution()


2.0.0


Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [0]:
def read_training_and_test_files():
  X_train = pd.read_csv('train_emoji.csv', names = ['X', 'Y'])
  X_test = pd.read_csv('test_emoji.csv', names = ['X', 'Y'])  
  return X_train, X_test
  
def read_files_and_retrieve_wordtovec_map(filename):
  X_train = pd.read_csv('train_emoji.csv', names = ['X', 'Y'])
  X_test = pd.read_csv('test_emoji.csv', names = ['X', 'Y'])
  words, word_to_vec_map = read_glove_vecs(filename)
  return X_train, X_test, word_to_vec_map

def process_training_and_test_values(X_train, X_test):
  X_train_X = X_train['X'].values.tolist()
  X_train_Y = X_train['Y'].values.tolist()

  X_test_X = X_test['X'].values.tolist()
  X_test_Y = X_test['Y'].values.tolist()

  X_train_lower = [x.lower() for x in X_train_X]
  X_test_lower = [x.lower() for x in X_test_X]
  return X_train_lower, X_test_lower, X_train_Y, X_test_Y
  
def tokenize_values(values):
  return [nltk.word_tokenize(item) for item in values]

def convert_to_nparray(values, value_type):
  return np.asarray(values).astype(value_type)



In [0]:
""" Converts sentences to average word embeddings. """
def sentence_to_avg(corpus, word_to_vec_map, vector_dim):
    avg = np.zeros((vector_dim,))
    averages = []
    for sentence in corpus:
      sum_val = 0
      for word in sentence:
        try:
          sum_val += word_to_vec_map[word]
        except:
          pass
      avg = sum_val/len(sentence)
      averages.append(avg)
    return averages

""" Converts sentences to min word embeddings """
def sentence_to_min(corpus, word_to_vec_map, vector_dim):
  minimum = []
  for sentence in corpus:
    min_val = np.ones((vector_dim,)) * np.inf
    for word in sentence:
      try:
        min_val = np.minimum(min_val, word_to_vec_map[word])
      except:
        pass
    minimum.append(min_val)
  return minimum

""" Converts sentences to max word embeddings"""
def sentence_to_max(corpus, word_to_vec_map, vector_dim):
  maximum = []
  for sentence in corpus:
    max_val = np.zeros((vector_dim,))
    for word in sentence:
      try:
        max_val = np.maximum(max_val, word_to_vec_map[word])
      except:
        pass
    maximum.append(max_val)
  return maximum

""" Converts sentences to word average embeddings"""
def combine_min_max_avg(corpus, word_to_vec_map, vector_dim):
  avg = sentence_to_avg(corpus, word_to_vec_map, vector_dim)
  minimum = sentence_to_min(corpus, word_to_vec_map, vector_dim)
  maximum = sentence_to_max(corpus, word_to_vec_map, vector_dim)    
  return list(zip(avg, minimum, maximum))
  
  

In [0]:
def Model_to_Fit(hidden_layers, classes):
  hidden_layer_size, num_classes = hidden_layers, classes
  layers = [
          tf.keras.layers.Dense(num_classes, activation='softmax')
      ]

  model = tf.keras.Sequential(layers)
  optimizer = tf.keras.optimizers.Adam(0.001)
  model.compile(optimizer= optimizer, loss='sparse_categorical_crossentropy', validation_split = 0.2,  metrics = ['accuracy'])
  return model

In [0]:
def CreateConfusionMatrixandReturnPredictions(actual_labels, predicted_labels):
  max_indices = []
  for row in predicted_labels:
    maximum_val = 0
    max_index = 0
    for index, value in enumerate(row):
      if (value > maximum_val):
        max_index =  index
        maximum_val = value
    max_indices.append(max_index)
  print(tf.math.confusion_matrix(actual_labels, max_indices, num_classes = 5))
  return max_indices

In [0]:
def ShowCorrectandIncorrectPredictions(test_label_set, pred_val):
  correctness_of_predictions = test_label_dataset == pred_val
  incorrect_values = []
  correct_values = []
  for i, w in enumerate(correctness_of_predictions):
    if(w == False):
      incorrect_values.append((test_label_dataset[i], pred_val[i], i))
    if(w == True):
      correct_values.append((test_label_dataset[i], pred_val[i], i))

  print("""----------------------------
Incorrect Predictions""")
  for i in incorrect_values:
    print(X_test_lower[i[2]], i[0], i[1])
  
  print("""-------------------------- 
Correct Predictions""")
  for i in correct_values:
    print(X_test_lower[i[2]], i[0], i[1])  

In [0]:
# Glove vector of 50 Dimensions for Twitter using Avg
X_train, X_test, word_to_vec_map = read_files_and_retrieve_wordtovec_map('glove.twitter.27B.50d.txt')
X_train_lower, X_test_lower, X_train_Y, X_test_Y = process_training_and_test_values(X_train, X_test)
tokenized_training_data = tokenize_values(X_train_lower)
tokenized_test_labels = tokenize_values(X_test_lower)

avg = sentence_to_avg(tokenized_training_data, word_to_vec_map, 50) 
test_avg = sentence_to_avg(tokenized_test_labels, word_to_vec_map, 50)


training_dataset = convert_to_nparray(avg, 'float32')
testing_dataset = convert_to_nparray(test_avg, 'float32')

training_label_dataset = convert_to_nparray(X_train_Y, 'float32')
test_label_dataset = convert_to_nparray(X_test_Y, 'float32')

model = Model_to_Fit(512, 5)
model.fit(x = training_dataset, y = training_label_dataset, epochs = 100)

predictions = model.predict(testing_dataset)
model.get_weights()
print(model.evaluate(testing_dataset, test_label_dataset))

pred_val = CreateConfusionMatrixandReturnPredictions(test_label_dataset, convert_to_nparray(predictions, 'float32'))
ShowCorrectandIncorrectPredictions(test_label_dataset, pred_val)

Train on 132 samples
Epoch 1/100
132/132 [==============================] - 1s 5ms/sample - loss: 2.5661 - accuracy: 0.1288
Epoch 2/100
132/132 [==============================] - 0s 152us/sample - loss: 2.4809 - accuracy: 0.1288
Epoch 3/100
132/132 [==============================] - 0s 187us/sample - loss: 2.4012 - accuracy: 0.1288
Epoch 4/100
132/132 [==============================] - 0s 120us/sample - loss: 2.3252 - accuracy: 0.1288
Epoch 5/100
132/132 [==============================] - 0s 125us/sample - loss: 2.2538 - accuracy: 0.1288
Epoch 6/100
132/132 [==============================] - 0s 142us/sample - loss: 2.1886 - accuracy: 0.1288
Epoch 7/100
132/132 [==============================] - 0s 138us/sample - loss: 2.1265 - accuracy: 0.1288
Epoch 8/100
132/132 [==============================] - 0s 108us/sample - loss: 2.0670 - accuracy: 0.1364
Epoch 9/100
132/132 [==============================] - 0s 109us/sample - loss: 2.0106 - accuracy: 0.1364
Epoch 10/100
132/132 [==============

In [0]:
# Glove vector of 50 Dimensions for Twitter using Min 
X_train, X_test, word_to_vec_map = read_files_and_retrieve_wordtovec_map('glove.twitter.27B.50d.txt')
X_train_lower, X_test_lower, X_train_Y, X_test_Y = process_training_and_test_values(X_train, X_test)
tokenized_training_data = tokenize_values(X_train_lower)
tokenized_test_labels = tokenize_values(X_test_lower)

minimum = sentence_to_min(tokenized_training_data, word_to_vec_map, 50) 
test_minimum = sentence_to_min(tokenized_test_labels, word_to_vec_map, 50)


training_dataset = convert_to_nparray(minimum, 'float32')
testing_dataset = convert_to_nparray(test_minimum, 'float32')

training_label_dataset = convert_to_nparray(X_train_Y, 'float32')
test_label_dataset = convert_to_nparray(X_test_Y, 'float32')

model = Model_to_Fit(512, 5)
model.fit(x = training_dataset, y = training_label_dataset, epochs = 100)

predictions = model.predict(testing_dataset)
pred_val = np.amax(convert_to_nparray(predictions, 'float32'), axis = 1)
model.get_weights()
print(model.evaluate(testing_dataset, test_label_dataset))

pred_val = CreateConfusionMatrixandReturnPredictions(test_label_dataset, convert_to_nparray(predictions, 'float32'))
ShowCorrectandIncorrectPredictions(test_label_dataset, pred_val)

Train on 132 samples
Epoch 1/100
132/132 [==============================] - 0s 2ms/sample - loss: 2.3069 - accuracy: 0.2803
Epoch 2/100
132/132 [==============================] - 0s 160us/sample - loss: 2.1812 - accuracy: 0.2879
Epoch 3/100
132/132 [==============================] - 0s 149us/sample - loss: 2.0771 - accuracy: 0.2955
Epoch 4/100
132/132 [==============================] - 0s 174us/sample - loss: 1.9761 - accuracy: 0.2955
Epoch 5/100
132/132 [==============================] - 0s 212us/sample - loss: 1.8987 - accuracy: 0.2879
Epoch 6/100
132/132 [==============================] - 0s 179us/sample - loss: 1.8309 - accuracy: 0.2727
Epoch 7/100
132/132 [==============================] - 0s 173us/sample - loss: 1.7752 - accuracy: 0.2652
Epoch 8/100
132/132 [==============================] - 0s 164us/sample - loss: 1.7276 - accuracy: 0.2576
Epoch 9/100
132/132 [==============================] - 0s 176us/sample - loss: 1.6933 - accuracy: 0.2652
Epoch 10/100
132/132 [==============

In [0]:
# Glove vector of 50 Dimensions for Twitter using Max
X_train, X_test, word_to_vec_map = read_files_and_retrieve_wordtovec_map('glove.twitter.27B.50d.txt')
X_train_lower, X_test_lower, X_train_Y, X_test_Y = process_training_and_test_values(X_train, X_test)
tokenized_training_data = tokenize_values(X_train_lower)
tokenized_test_labels = tokenize_values(X_test_lower)

maximum = sentence_to_max(tokenized_training_data, word_to_vec_map, 50) 
test_maximum = sentence_to_max(tokenized_test_labels, word_to_vec_map, 50)


training_dataset = convert_to_nparray(maximum, 'float32')
testing_dataset = convert_to_nparray(test_maximum, 'float32')

training_label_dataset = convert_to_nparray(X_train_Y, 'float32')
test_label_dataset = convert_to_nparray(X_test_Y, 'float32')

model = Model_to_Fit(512, 5)
model.fit(x = training_dataset, y = training_label_dataset, epochs = 100)

predictions = model.predict(testing_dataset)
pred_val = np.amax(convert_to_nparray(predictions, 'float32'), axis = 1)
model.get_weights()
model.evaluate(testing_dataset, test_label_dataset)

pred_val = CreateConfusionMatrixandReturnPredictions(test_label_dataset, convert_to_nparray(predictions, 'float32'))
ShowCorrectandIncorrectPredictions(test_label_dataset, pred_val)

Train on 132 samples
Epoch 1/100
132/132 [==============================] - 0s 4ms/sample - loss: 1.8973 - accuracy: 0.1667
Epoch 2/100
132/132 [==============================] - 0s 117us/sample - loss: 1.8109 - accuracy: 0.2045
Epoch 3/100
132/132 [==============================] - 0s 108us/sample - loss: 1.7375 - accuracy: 0.2273
Epoch 4/100
132/132 [==============================] - 0s 128us/sample - loss: 1.6805 - accuracy: 0.2424
Epoch 5/100
132/132 [==============================] - 0s 102us/sample - loss: 1.6301 - accuracy: 0.2500
Epoch 6/100
132/132 [==============================] - 0s 103us/sample - loss: 1.5858 - accuracy: 0.2652
Epoch 7/100
132/132 [==============================] - 0s 104us/sample - loss: 1.5533 - accuracy: 0.2955
Epoch 8/100
132/132 [==============================] - 0s 115us/sample - loss: 1.5276 - accuracy: 0.3712
Epoch 9/100
132/132 [==============================] - 0s 118us/sample - loss: 1.5032 - accuracy: 0.4091
Epoch 10/100
132/132 [==============

<tf.Tensor: id=7443, shape=(5, 5), dtype=int32, numpy=
array([[ 7,  0,  0,  0,  0],
       [ 8,  0,  0,  0,  0],
       [18,  0,  0,  0,  0],
       [16,  0,  0,  0,  0],
       [ 7,  0,  0,  0,  0]], dtype=int32)>

In [0]:
# Glove vector of 50 Dimensions for Twitter using Max-Min-Avg 
X_train, X_test, word_to_vec_map = read_files_and_retrieve_wordtovec_map('glove.twitter.27B.50d.txt')
X_train_lower, X_test_lower, X_train_Y, X_test_Y = process_training_and_test_values(X_train, X_test)
tokenized_training_data = tokenize_values(X_train_lower)
tokenized_test_labels = tokenize_values(X_test_lower)

combined_dataset = combine_min_max_avg(tokenized_training_data, word_to_vec_map, 50)
test_combined_dataset = combine_min_max_avg(tokenized_test_labels, word_to_vec_map, 50)


training_dataset = convert_to_nparray(combined_dataset, 'float32')
testing_dataset = convert_to_nparray(test_combined_dataset, 'float32')

training_label_dataset = convert_to_nparray(X_train_Y, 'float32')
test_label_dataset = convert_to_nparray(X_test_Y, 'float32')

train = []
for i in training_dataset:
  train.append(i.flatten())
  
test = []
for i in testing_dataset:
  test.append(i.flatten())
  
test_val = convert_to_nparray(test, 'float32')

train_val = convert_to_nparray(train, 'float32')
print(train_val.shape)

model = Model_to_Fit(512, 5)
model.fit(x = train_val, y = training_label_dataset, epochs = 100)

predictions = model.predict(test_val)
print(model.evaluate(test_val, test_label_dataset))

pred_val = CreateConfusionMatrixandReturnPredictions(test_label_dataset, convert_to_nparray(predictions, 'float32'))
ShowCorrectandIncorrectPredictions(test_label_dataset, pred_val)

(132, 150)
Train on 132 samples
Epoch 1/100
132/132 [==============================] - 0s 2ms/sample - loss: 2.0402 - accuracy: 0.2576
Epoch 2/100
132/132 [==============================] - 0s 116us/sample - loss: 1.8339 - accuracy: 0.2727
Epoch 3/100
132/132 [==============================] - 0s 104us/sample - loss: 1.6900 - accuracy: 0.3030
Epoch 4/100
132/132 [==============================] - 0s 121us/sample - loss: 1.6049 - accuracy: 0.3561
Epoch 5/100
132/132 [==============================] - 0s 120us/sample - loss: 1.5629 - accuracy: 0.3409
Epoch 6/100
132/132 [==============================] - 0s 109us/sample - loss: 1.5331 - accuracy: 0.3485
Epoch 7/100
132/132 [==============================] - 0s 104us/sample - loss: 1.5042 - accuracy: 0.3333
Epoch 8/100
132/132 [==============================] - 0s 142us/sample - loss: 1.4765 - accuracy: 0.3636
Epoch 9/100
132/132 [==============================] - 0s 94us/sample - loss: 1.4466 - accuracy: 0.3939
Epoch 10/100
132/132 [====

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Word2Vec of 300 dimensions on Google News using avg
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  
X_train, X_test = read_training_and_test_files()
X_train_lower, X_test_lower, X_train_Y, X_test_Y = process_training_and_test_values(X_train, X_test)
tokenized_training_data = tokenize_values(X_train_lower)
tokenized_test_labels = tokenize_values(X_test_lower)


avg = sentence_to_avg(tokenized_training_data, model, 300) 
test_avg = sentence_to_avg(tokenized_test_labels, model, 300)

training_dataset = convert_to_nparray(avg, 'float32')
testing_dataset = convert_to_nparray(test_avg, 'float32')

training_label_dataset = convert_to_nparray(X_train_Y, 'float32')
test_label_dataset = convert_to_nparray(X_test_Y, 'float32')

model = Model_to_Fit(512, 5)
model.fit(x = training_dataset, y = training_label_dataset, epochs = 100)

predictions = model.predict(testing_dataset)
pred_val = np.amax(convert_to_nparray(predictions, 'float32'), axis = 1)
model.get_weights()
model.evaluate(testing_dataset, test_label_dataset)

#tf.math.confusion_matrix(test_label_dataset, pred_val, num_classes = 5)
pred_val = CreateConfusionMatrixandReturnPredictions(test_label_dataset, convert_to_nparray(predictions, 'float32'))
ShowCorrectandIncorrectPredictions(test_label_dataset, pred_val)

56/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [18]:
# Glove vector of 300 Dimensions on Wikipedia using Avg
X_train, X_test, word_to_vec_map = read_files_and_retrieve_wordtovec_map('glove.6B.300d.txt')
X_train_lower, X_test_lower, X_train_Y, X_test_Y = process_training_and_test_values(X_train, X_test)
tokenized_training_data = tokenize_values(X_train_lower)
tokenized_test_labels = tokenize_values(X_test_lower)

avg = sentence_to_avg(tokenized_training_data, word_to_vec_map, 300) 
test_avg = sentence_to_avg(tokenized_test_labels, word_to_vec_map, 300)


training_dataset = convert_to_nparray(avg, 'float32')
testing_dataset = convert_to_nparray(test_avg, 'float32')

training_label_dataset = convert_to_nparray(X_train_Y, 'float32')
test_label_dataset = convert_to_nparray(X_test_Y, 'float32')

model = Model_to_Fit(512, 5)
model.fit(x = training_dataset, y = training_label_dataset, epochs = 100)

predictions = model.predict(testing_dataset)
pred_val = np.amax(convert_to_nparray(predictions, 'float32'), axis = 1)
model.get_weights()
model.evaluate(testing_dataset, test_label_dataset)

#tf.math.confusion_matrix(test_label_dataset, pred_val, num_classes = 5)
pred_val = CreateConfusionMatrixandReturnPredictions(test_label_dataset, convert_to_nparray(predictions, 'float32'))
ShowCorrectandIncorrectPredictions(test_label_dataset, pred_val)

Train on 132 samples
Epoch 1/100
132/132 [==============================] - 0s 2ms/sample - loss: 1.5720 - accuracy: 0.2424
Epoch 2/100
132/132 [==============================] - 0s 142us/sample - loss: 1.5284 - accuracy: 0.3106
Epoch 3/100
132/132 [==============================] - 0s 138us/sample - loss: 1.4972 - accuracy: 0.3561
Epoch 4/100
132/132 [==============================] - 0s 141us/sample - loss: 1.4663 - accuracy: 0.3636
Epoch 5/100
132/132 [==============================] - 0s 131us/sample - loss: 1.4380 - accuracy: 0.3788
Epoch 6/100
132/132 [==============================] - 0s 144us/sample - loss: 1.4123 - accuracy: 0.4394
Epoch 7/100
132/132 [==============================] - 0s 120us/sample - loss: 1.3883 - accuracy: 0.4470
Epoch 8/100
132/132 [==============================] - 0s 141us/sample - loss: 1.3604 - accuracy: 0.4470
Epoch 9/100
132/132 [==============================] - 0s 128us/sample - loss: 1.3351 - accuracy: 0.4545
Epoch 10/100
132/132 [==============

In [0]:
# Glove vector of 50 Dimensions on Wikipedia using avg
X_train, X_test, word_to_vec_map = read_files_and_retrieve_wordtovec_map('glove.6B.50d.txt')
X_train_lower, X_test_lower, X_train_Y, X_test_Y = process_training_and_test_values(X_train, X_test)
tokenized_training_data = tokenize_values(X_train_lower)
tokenized_test_labels = tokenize_values(X_test_lower)

avg = sentence_to_avg(tokenized_training_data, word_to_vec_map, 50) 
test_avg = sentence_to_avg(tokenized_test_labels, word_to_vec_map, 50)


training_dataset = convert_to_nparray(avg, 'float32')
testing_dataset = convert_to_nparray(test_avg, 'float32')

training_label_dataset = convert_to_nparray(X_train_Y, 'float32')
test_label_dataset = convert_to_nparray(X_test_Y, 'float32')

model = Model_to_Fit(512, 5)
model.fit(x = training_dataset, y = training_label_dataset, epochs = 100)

predictions = model.predict(testing_dataset)
pred_val = convert_to_nparray(predictions, 'float32')
#model.get_weights()
model.evaluate(testing_dataset, test_label_dataset)

#tf.math.confusion_matrix(test_label_dataset, pred_val, num_classes = 5)
pred_val = CreateConfusionMatrixandReturnPredictions(test_label_dataset, convert_to_nparray(predictions, 'float32'))
ShowCorrectandIncorrectPredictions(test_label_dataset, pred_val)

Train on 132 samples
Epoch 1/100
132/132 [==============================] - 0s 2ms/sample - loss: 1.8604 - accuracy: 0.1818
Epoch 2/100
132/132 [==============================] - 0s 116us/sample - loss: 1.8098 - accuracy: 0.1970
Epoch 3/100
132/132 [==============================] - 0s 129us/sample - loss: 1.7695 - accuracy: 0.1970
Epoch 4/100
132/132 [==============================] - 0s 140us/sample - loss: 1.7337 - accuracy: 0.1894
Epoch 5/100
132/132 [==============================] - 0s 133us/sample - loss: 1.7005 - accuracy: 0.1970
Epoch 6/100
132/132 [==============================] - 0s 108us/sample - loss: 1.6735 - accuracy: 0.2121
Epoch 7/100
132/132 [==============================] - 0s 116us/sample - loss: 1.6491 - accuracy: 0.2273
Epoch 8/100
132/132 [==============================] - 0s 106us/sample - loss: 1.6297 - accuracy: 0.2348
Epoch 9/100
132/132 [==============================] - 0s 120us/sample - loss: 1.6090 - accuracy: 0.2348
Epoch 10/100
132/132 [==============

In [0]:
# Glove vector of 100 Dimensions on Wikipedia using avg
X_train, X_test, word_to_vec_map = read_files_and_retrieve_wordtovec_map('glove.6B.100d.txt')
X_train_lower, X_test_lower, X_train_Y, X_test_Y = process_training_and_test_values(X_train, X_test)
tokenized_training_data = tokenize_values(X_train_lower)
tokenized_test_labels = tokenize_values(X_test_lower)

avg = sentence_to_avg(tokenized_training_data, word_to_vec_map, 100) 
test_avg = sentence_to_avg(tokenized_test_labels, word_to_vec_map, 100)


training_dataset = convert_to_nparray(avg, 'float32')
testing_dataset = convert_to_nparray(test_avg, 'float32')

training_label_dataset = convert_to_nparray(X_train_Y, 'float32')
test_label_dataset = convert_to_nparray(X_test_Y, 'float32')

model = Model_to_Fit(512, 5)
model.fit(x = training_dataset, y = training_label_dataset, epochs = 100)

predictions = model.predict(testing_dataset)
pred_val = np.max(convert_to_nparray(predictions, 'float32'), axis = 1)

p = np.around(pred_val)
model.get_weights()
model.evaluate(testing_dataset, test_label_dataset)

#tf.math.confusion_matrix(test_label_dataset, p, num_classes = 5)
pred_val = CreateConfusionMatrixandReturnPredictions(test_label_dataset, convert_to_nparray(predictions, 'float32'))
ShowCorrectandIncorrectPredictions(test_label_dataset, pred_val)

Train on 132 samples
Epoch 1/100
132/132 [==============================] - 0s 2ms/sample - loss: 1.5761 - acc: 0.2727
Epoch 2/100
132/132 [==============================] - 0s 153us/sample - loss: 1.3252 - acc: 0.4015
Epoch 3/100
132/132 [==============================] - 0s 136us/sample - loss: 1.1413 - acc: 0.6515
Epoch 4/100
132/132 [==============================] - 0s 136us/sample - loss: 1.0171 - acc: 0.7652
Epoch 5/100
132/132 [==============================] - 0s 153us/sample - loss: 0.9345 - acc: 0.7121
Epoch 6/100
132/132 [==============================] - 0s 150us/sample - loss: 0.8599 - acc: 0.7652
Epoch 7/100
132/132 [==============================] - 0s 156us/sample - loss: 0.7894 - acc: 0.8030
Epoch 8/100
132/132 [==============================] - 0s 161us/sample - loss: 0.7696 - acc: 0.7348
Epoch 9/100
132/132 [==============================] - 0s 147us/sample - loss: 0.6960 - acc: 0.7879
Epoch 10/100
132/132 [==============================] - 0s 157us/sample - loss: 0

<tf.Tensor: id=54999, shape=(5, 5), dtype=int32, numpy=
array([[ 2,  5,  0,  0,  0],
       [ 0,  8,  0,  0,  0],
       [ 0, 18,  0,  0,  0],
       [ 0, 16,  0,  0,  0],
       [ 0,  7,  0,  0,  0]], dtype=int32)>

In [19]:
# Glove vector of 100 Dimensions for Twitter using Avg
X_train, X_test, word_to_vec_map = read_files_and_retrieve_wordtovec_map('glove.twitter.27B.100d.txt')
X_train_lower, X_test_lower, X_train_Y, X_test_Y = process_training_and_test_values(X_train, X_test)
tokenized_training_data = tokenize_values(X_train_lower)
tokenized_test_labels = tokenize_values(X_test_lower)

avg = sentence_to_avg(tokenized_training_data, word_to_vec_map, 100) 
test_avg = sentence_to_avg(tokenized_test_labels, word_to_vec_map, 100)


training_dataset = convert_to_nparray(avg, 'float32')
testing_dataset = convert_to_nparray(test_avg, 'float32')

training_label_dataset = convert_to_nparray(X_train_Y, 'float32')
test_label_dataset = convert_to_nparray(X_test_Y, 'float32')

model = Model_to_Fit(512, 5)
model.fit(x = training_dataset, y = training_label_dataset, epochs = 100)

predictions = model.predict(testing_dataset)
model.get_weights()
print(model.evaluate(testing_dataset, test_label_dataset))

pred_val = CreateConfusionMatrixandReturnPredictions(test_label_dataset, convert_to_nparray(predictions, 'float32'))
ShowCorrectandIncorrectPredictions(test_label_dataset, pred_val)

Train on 132 samples
Epoch 1/100
132/132 [==============================] - 0s 2ms/sample - loss: 2.0385 - accuracy: 0.1439
Epoch 2/100
132/132 [==============================] - 0s 147us/sample - loss: 1.9674 - accuracy: 0.1364
Epoch 3/100
132/132 [==============================] - 0s 185us/sample - loss: 1.9066 - accuracy: 0.1364
Epoch 4/100
132/132 [==============================] - 0s 132us/sample - loss: 1.8493 - accuracy: 0.1364
Epoch 5/100
132/132 [==============================] - 0s 308us/sample - loss: 1.7968 - accuracy: 0.1515
Epoch 6/100
132/132 [==============================] - 0s 145us/sample - loss: 1.7491 - accuracy: 0.1667
Epoch 7/100
132/132 [==============================] - 0s 147us/sample - loss: 1.7090 - accuracy: 0.1591
Epoch 8/100
132/132 [==============================] - 0s 168us/sample - loss: 1.6686 - accuracy: 0.1667
Epoch 9/100
132/132 [==============================] - 0s 178us/sample - loss: 1.6326 - accuracy: 0.1818
Epoch 10/100
132/132 [==============

In [0]:
# Glove vector of 200 Dimensions on Wikipedia using avg
X_train, X_test, word_to_vec_map = read_files_and_retrieve_wordtovec_map('glove.6B.200d.txt')
X_train_lower, X_test_lower, X_train_Y, X_test_Y = process_training_and_test_values(X_train, X_test)
tokenized_training_data = tokenize_values(X_train_lower)
tokenized_test_labels = tokenize_values(X_test_lower)

avg = sentence_to_avg(tokenized_training_data, word_to_vec_map, 200) 
test_avg = sentence_to_avg(tokenized_test_labels, word_to_vec_map, 200)

training_dataset = convert_to_nparray(avg, 'float32')
testing_dataset = convert_to_nparray(test_avg, 'float32')

training_label_dataset = convert_to_nparray(X_train_Y, 'float32')
test_label_dataset = convert_to_nparray(X_test_Y, 'float32')

model = Model_to_Fit(512, 5)
model.fit(x = training_dataset, y = training_label_dataset, epochs = 100)

predictions = model.predict(testing_dataset)
model.get_weights()
print(model.evaluate(testing_dataset, test_label_dataset))

pred_val = CreateConfusionMatrixandReturnPredictions(test_label_dataset, convert_to_nparray(predictions, 'float32'))
ShowCorrectandIncorrectPredictions(test_label_dataset, pred_val)

Train on 132 samples
Epoch 1/100
132/132 [==============================] - 0s 2ms/sample - loss: 1.6287 - acc: 0.2197
Epoch 2/100
132/132 [==============================] - 0s 123us/sample - loss: 1.5736 - acc: 0.2121
Epoch 3/100
132/132 [==============================] - 0s 138us/sample - loss: 1.5316 - acc: 0.2955
Epoch 4/100
132/132 [==============================] - 0s 130us/sample - loss: 1.5004 - acc: 0.3409
Epoch 5/100
132/132 [==============================] - 0s 139us/sample - loss: 1.4733 - acc: 0.3561
Epoch 6/100
132/132 [==============================] - 0s 129us/sample - loss: 1.4492 - acc: 0.3788
Epoch 7/100
132/132 [==============================] - 0s 127us/sample - loss: 1.4270 - acc: 0.3939
Epoch 8/100
132/132 [==============================] - 0s 140us/sample - loss: 1.4064 - acc: 0.4091
Epoch 9/100
132/132 [==============================] - 0s 131us/sample - loss: 1.3849 - acc: 0.4318
Epoch 10/100
132/132 [==============================] - 0s 126us/sample - loss: 1

In [22]:
# Glove vector of 200 Dimensions for Twitter using Avg
X_train, X_test, word_to_vec_map = read_files_and_retrieve_wordtovec_map('glove.twitter.27B.200d.txt')
X_train_lower, X_test_lower, X_train_Y, X_test_Y = process_training_and_test_values(X_train, X_test)
tokenized_training_data = tokenize_values(X_train_lower)
tokenized_test_labels = tokenize_values(X_test_lower)

avg = sentence_to_avg(tokenized_training_data, word_to_vec_map, 200) 
test_avg = sentence_to_avg(tokenized_test_labels, word_to_vec_map, 200)


training_dataset = convert_to_nparray(avg, 'float32')
testing_dataset = convert_to_nparray(test_avg, 'float32')

training_label_dataset = convert_to_nparray(X_train_Y, 'float32')
test_label_dataset = convert_to_nparray(X_test_Y, 'float32')

model = Model_to_Fit(512, 5)
model.fit(x = training_dataset, y = training_label_dataset, epochs = 100)

predictions = model.predict(testing_dataset)
model.get_weights()
print(model.evaluate(testing_dataset, test_label_dataset))

pred_val = CreateConfusionMatrixandReturnPredictions(test_label_dataset, convert_to_nparray(predictions, 'float32'))
ShowCorrectandIncorrectPredictions(test_label_dataset, pred_val)

Train on 132 samples
Epoch 1/100
132/132 [==============================] - 0s 2ms/sample - loss: 1.7316 - accuracy: 0.2045
Epoch 2/100
132/132 [==============================] - 0s 123us/sample - loss: 1.6696 - accuracy: 0.2045
Epoch 3/100
132/132 [==============================] - 0s 135us/sample - loss: 1.6199 - accuracy: 0.2197
Epoch 4/100
132/132 [==============================] - 0s 116us/sample - loss: 1.5729 - accuracy: 0.2273
Epoch 5/100
132/132 [==============================] - 0s 125us/sample - loss: 1.5349 - accuracy: 0.2424
Epoch 6/100
132/132 [==============================] - 0s 121us/sample - loss: 1.4966 - accuracy: 0.3030
Epoch 7/100
132/132 [==============================] - 0s 126us/sample - loss: 1.4634 - accuracy: 0.3409
Epoch 8/100
132/132 [==============================] - 0s 111us/sample - loss: 1.4335 - accuracy: 0.3788
Epoch 9/100
132/132 [==============================] - 0s 116us/sample - loss: 1.4063 - accuracy: 0.4318
Epoch 10/100
132/132 [==============